In [1]:
pip install --upgrade langchain langchain-core langchain-community langchain-google-genai google-generativeai

  Using cached langchain_google_genai-2.1.9-py3-none-any.whl.metadata (7.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pro

In [2]:
!pip install gradio langchain langchain-core langchain-community langchain-google-genai google-generativeai

In [3]:
!pip install duckduckgo-search



In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

In [12]:
import os
import gradio as gr
from langchain_core.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from dotenv import load_dotenv
import time

# Load variables from .env
load_dotenv()

# ✅ Replace with your actual Gemini API key
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# ✅ Gemini model for math, logic, code
llm_gemini = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.3, google_api_key=api_key) # Pass api_key here

# ✅ DuckDuckGo for general questions
duckduckgo_tool = DuckDuckGoSearchRun()

# ✅ Gemini Math Tool Function
def gemini_math_tool_func(query: str) -> str:
    response = llm_gemini.invoke(query)
    return response.content

# ✅ Define tools
tools = [
    Tool(
        name="DuckDuckGo",
        func=duckduckgo_tool.run,
        description="Use for general questions or real-world facts."
    ),
    Tool(
        name="GeminiMath",
        func=gemini_math_tool_func,
        description="Use for math, logic, and reasoning problems."
    )
]

# ✅ Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm_gemini,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Gradio Interface Function
def ask_question(query):
    max_retries = 2  # Maximum number of retries
    retry_delay = 46  # Delay in seconds
    for attempt in range(max_retries + 1):
        try:
            return agent.invoke(query)
        except Exception as e:
            if "429" in str(e):
                if attempt < max_retries:
                    print(f"Rate limit exceeded. Retrying after {retry_delay} seconds (attempt {attempt + 1}/{max_retries})...")
                    time.sleep(retry_delay)
                else:
                    return f"❌ Error: Rate limit exceeded after {max_retries} retries."
            else:
                return f"❌ Error: {e}"
    return "❌ Error: Max retries reached."

# ✅ Build the Web Interface
interface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
    outputs="text",
    title="🧠 Ask Anything: Gemini + DuckDuckGo",
    description="✔ General questions go to DuckDuckGo, math or logic to Gemini"
)

# ✅ Launch the App
interface.launch()

c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=3916, family=2, type=1, proto=0>
  s = socket.socket()
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=3076, family=2, type=1, proto=0>
  s = socket.socket()
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=3916, family=2, type=1, proto=0>
  s = socket.socket()
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=3076, family=2, type=1, proto=0>
  s = socket.socket()
c:\Users\VICTUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=3916, family=2, type=1, proto=0>
  s = socket.socket()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
].


Rate limit exceeded. Retrying after 46 seconds (attempt 1/2)...


> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 38
}
].


Rate limit exceeded. Retrying after 46 seconds (attempt 2/2)...


> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 49
}
].
